In [1]:
from pathlib import Path
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import imgaug.augmenters as iaa
import numpy as np
import matplotlib.pyplot as plt
from dataset import BrainDataset
from model import UNet

augment_pipeline = iaa.Sequential([
        iaa.Affine(scale=(0.85, 1.15), rotate=(-45, 45)),
        iaa.ElasticTransformation()
        ])

train_path = Path('Preprocessed/train/')
val_path = Path('Preprocessed/val/')

train_dataset = BrainDataset(train_path, augment_pipeline)
val_dataset = BrainDataset(val_path, None)

batch_size = 8
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

class DiceLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, pred_mask, actual_mask):
        pred = torch.flatten(pred_mask)
        actual = torch.flatten(actual_mask)
        
        numerator = (pred * actual).sum()
        denominator = pred.sum() + actual.sum() + 1e-8
        dice_score = (2 * numerator) / denominator
        return 1 - dice_score
        
class BrainSegmentation(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = UNet()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.loss_function = DiceLoss()
        
    def forward(self, data):
        return torch.sigmoid(self.model(data))
    
    def training_step(self, batch, batch_idx):
        slice_data, slice_mask = batch
        slice_mask = slice_mask.float()
        pred_mask = self(slice_data)
        loss = self.loss_function(pred_mask, slice_mask)
        self.log('Train Loss', loss)
        
        if batch_idx % 50 == 0:
            self.log_images(slice_data.cpu(), pred_mask.cpu(), slice_mask.cpu(), 'Train')
            
        return loss

    def validation_step(self, batch, batch_idx):
        slice_data, slice_mask = batch
        slice_mask = slice_mask.float()
        pred_mask = self(slice_data)
        loss = self.loss_function(pred_mask, slice_mask)
        self.log('Val Loss', loss)
        
        if batch_idx % 2 == 0:
            self.log_images(slice_data.cpu(), pred_mask.cpu(), slice_mask.cpu(), 'Val')
            
        return loss
        
    def log_images(self, slice_data, pred_mask, slice_mask, name):
        pred_mask = pred_mask > 0.5
        fig, axis = plt.subplots(1, 2)
        
        axis[0].imshow(slice_data[0][0], cmap='bone')
        actual_mask_ma = np.ma.masked_where(slice_mask[0][0] == 0, slice_mask[0][0])
        axis[0].imshow(actual_mask_ma, alpha=0.6)
        
        axis[1].imshow(slice_data[0][0], cmap='bone')
        pred_mask_ma = np.ma.masked_where(pred_mask[0][0] == 0, pred_mask[0][0])
        axis[1].imshow(pred_mask_ma, alpha=0.6)
        
        self.logger.experiment.add_figure(name, fig, self.global_step)
        
    def configure_optimizers(self):
        return [self.optimizer]

In [ ]:
# Initialize same random values as weights
torch.manual_seed(0)
model = BrainSegmentation()

checkpoint_callback = ModelCheckpoint(monitor='Val Loss', save_top_k=10, mode='min')

# Assign 0 for CPU training
gpus = 1
trainer = pl.Trainer(gpus=gpus, logger=TensorBoardLogger(save_dir='./logs'),
                     log_every_n_steps=1, callbacks=checkpoint_callback, max_epochs=50)

if __name__ == '__main__':
    trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: ./logs\default

  | Name          | Type     | Params
-------------------------------------------
0 | model         | UNet     | 487 K 
1 | loss_function | DiceLoss | 0     
-------------------------------------------
487 K     Trainable params
0         Non-trainable params
487 K     Total params
1.948     Total estimated model params size (MB)


Validation sanity check:   0%|                                                                   | 0/2 [00:00<?, ?it/s]

C:\Users\BILGISAYAR\Anaconda3\envs\pytorchenv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:117: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
C:\Users\BILGISAYAR\Anaconda3\envs\pytorchenv\lib\site-packages\torch\nn\functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
C:\Users\BILGISAYAR\Anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\image.py:446: UserWarning: Warning: converting a masked element to nan.
  dv = np.flo

C:\Users\BILGISAYAR\Anaconda3\envs\pytorchenv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:117: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 0:  75%|████████████████████████████████████            | 1800/2400 [11:15<03:45,  2.66it/s, loss=0.173, v_num=0]
Validating: 0it [00:00, ?it/s]
Validating:  14%|█████████▍                                                           | 82/600 [00:19<01:54,  4.54it/s]


...

Epoch 20:  91%|█████████████████████████████████████████▉    | 2189/2400 [30:49<02:58,  1.18it/s, loss=0.0166, v_num=0]


Epoch 20:  94%|███████████████████████████████████████████▏  | 2255/2400 [31:10<02:00,  1.21it/s, loss=0.0166, v_num=0]


Epoch 20:  97%|████████████████████████████████████████████▍ | 2321/2400 [31:32<01:04,  1.23it/s, loss=0.0166, v_num=0]


Epoch 20:  99%|█████████████████████████████████████████████▊| 2387/2400 [31:53<00:10,  1.25it/s, loss=0.0166, v_num=0]


Epoch 21:   4%|██                                             | 108/2400 [01:37<34:37,  1.10it/s, loss=0.0187, v_num=0]